In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("../raw_data/data_clean.csv")

In [3]:
data.head(3)

,steam_appid,name,release_date,english,developer,publisher,platforms,categories,genres,steamspy_tags,...,wrestling,zombies,e_sports,owner_median,owner_lower,owner_upper,has_a_website,nb_review,owner_estimated,revenu
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,...,0,0,550,15000000.0,10000000,20000000,0,127873,15000000.0,107850000.0
1,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,...,0,0,0,7500000.0,5000000,10000000,1,3814,7500000.0,29925000.0
2,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,...,0,0,0,7500000.0,5000000,10000000,0,5538,7500000.0,29925000.0


In [4]:
obj = data.isnull().sum()
for key,value in obj.iteritems():
    print(f'"{key}"')

"steam_appid"
"name"
"release_date"
"english"
"developer"
"publisher"
"platforms"
"categories"
"genres"
"steamspy_tags"
"achievements"
"positive_ratings"
"negative_ratings"
"average_playtime"
"median_playtime"
"owners"
"price"
"detailed_description"
"about_the_game"
"short_description"
"header_image"
"screenshots"
"background"
"website"
"1980s"
"1990s"
"2.5d"
"2d"
"2d_fighter"
"360_video"
"3d"
"3d_platformer"
"3d_vision"
"4_player_local"
"4x"
"6dof"
"atv"
"abstract"
"action"
"action_rpg"
"action_adventure"
"addictive"
"adventure"
"agriculture"
"aliens"
"alternate_history"
"america"
"animation_&_modeling"
"anime"
"arcade"
"arena_shooter"
"artificial_intelligence"
"assassin"
"asynchronous_multiplayer"
"atmospheric"
"audio_production"
"bmx"
"base_building"
"baseball"
"based_on_a_novel"
"basketball"
"batman"
"battle_royale"
"beat_em_up"
"beautiful"
"benchmark"
"bikes"
"blood"
"board_game"
"bowling"
"building"
"bullet_hell"
"bullet_time"
"crpg"
"capitalism"
"card_game"
"cartoon"
"cartoony"


In [5]:
X = data.drop(columns=["steam_appid",
"name",
"developer",
"publisher",
"platforms",
"categories",
"genres",
"steamspy_tags",
"achievements",
"positive_ratings",
"negative_ratings",
"average_playtime",
"median_playtime",
"owners",
"release_date",
"detailed_description",
"about_the_game",
"short_description",
"header_image",
"screenshots",
"background",
"website",
"owner_median",
"owner_lower",
"owner_upper",
"nb_review",
"owner_estimated",
"revenu"])
#X = data[['name']]
y = data['owner_estimated']

In [6]:
X

,english,price,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,...,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports,has_a_website
0,1,7.19,144,564,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,550,0
1,1,3.99,0,0,0,0,0,0,0,0,...,0,0,0,0,5,122,0,0,0,1
2,1,3.99,0,77,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,7.19,0,376,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,7.19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26862,1,2.09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26863,1,1.69,0,0,0,0,0,0,0,12,...,0,0,0,0,0,0,0,0,0,0
26864,1,3.99,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26865,1,5.19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# ohe = OneHotEncoder(sparse = False)
# ohe.fit(data[['owners']])
# y_encoded = ohe.transform(data[['owners']])

In [8]:
# ohe = OneHotEncoder(sparse = False)
# ohe.fit(data[['name']])
# X_encoded = ohe.transform(data[['name']])

In [9]:
# X_encoded = pd.DataFrame(X_encoded)

In [10]:
# X_encoded

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
model = KNeighborsRegressor()

In [13]:
model.fit(X_train, y_train)

KNeighborsRegressor()

In [14]:
cv_results = cross_validate(model, X_test, y_test, cv=5)

In [15]:
cv_results['test_score']

array([0.56502668, 0.53267462, 0.21060264, 0.52257582, 0.83950224])

In [16]:
model.cross_val_score(X_test, y_test)

AttributeError: 'KNeighborsRegressor' object has no attribute 'cross_val_score'

In [ ]:
# Preprocessor (prepare the dataset)
num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
cat_transformer = OneHotEncoder()

preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64'])),
    (cat_transformer, make_column_selector(dtype_include=['object','bool'])),
    remainder='passthrough')

# Add Estimator
pipe = make_pipeline(preproc, model)

# Train pipeline
pipe.fit(X_train,y_train)

# Make predictions
pipe.predict(X_test.iloc[0:2])

# Score model
pipe.score(X_test,y_test)

# Cross validate pipeline
cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2').mean()